The goal of this note book is to create a function to get the loans from a pool number 

In [118]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import os
from dotenv import load_dotenv

# Load environment variables from a .env file
from calendar import c
from math import log
from multiprocessing import pool
from selenium.webdriver.common.action_chains import ActionChains  # Add this import

url = "https://caweb.sba.gov/cls/dsp_login.cfm"

In [119]:
from re import S
from time import sleep


def login(username, password, driver):
    input_field = driver.find_element(By.ID, "OrigUserName")

    # Clear the field (optional)
    input_field.clear()

    # Send text to the input field
    input_field.send_keys(username)
    sleep(0.9)
    password_field = driver.find_element(By.ID, "OrigPassword")
    password_field.clear()
    password_field.send_keys(password)
    sleep(1)
    checkbox = driver.find_element(By.ID, "Answer")

    # Click the checkbox to check it
    if not checkbox.is_selected():  # Check if it's not already selected
        checkbox.click()
    sleep(1.1)
    submit_button = driver.find_element(By.ID, "SubmitButton")
    submit_button.click()
    sleep(3)

In [120]:
from selenium.webdriver.support.ui import Select


def get_pool_list(driver):
    driver.switch_to.frame("filecontainer")
    browse_menu = driver.find_element(By.XPATH, "/html/body/div/nav/ul/li[3]")

    # Create an action chain to hover over the "Browse" menu to reveal the submenu
    action = ActionChains(driver)
    action.move_to_element(browse_menu).perform()
    sleep(1)
    # Now, after hovering, find the "Pool" link
    pool_link = driver.find_element(
        By.XPATH, "//a[contains(@href, 'pfg_rpt_sba_pool_loan_opt')]"
    )
    pool_link.click()
    sleep(3)
    radio = driver.find_element(By.ID, "radio2")

    # Click the checkbox to check it
    if not radio.is_selected():  # Check if it's not already selected
        radio.click()
    month_dropdown = driver.find_element(By.ID, "fromMonth")

    # Create a Select object for the month dropdown
    select_month = Select(month_dropdown)

    # Set the month dropdown to the first option (1)
    select_month.select_by_value("1")
    sleep(2)
    month_dropdown = driver.find_element(By.ID, "toMonth")

    # Create a Select object for the month dropdown
    select_month = Select(month_dropdown)

    # Set the month dropdown to the first option (1)
    select_month.select_by_value("12")
    sleep(2)
    holder_list = driver.find_element(By.ID, "holderName")

    # Create a Select object for the month dropdown
    holder = Select(holder_list)

    # Set the month dropdown to the first option (1)
    holder.select_by_value("0")
    sleep(1)
    submit_button = driver.find_element(
        By.XPATH, "/html/body/div/div[2]/div/form/p[6]/input"
    )
    submit_button.click()
    sleep(5)
    table = driver.find_element(By.ID, "sbaPoolsReportTable")

    # Get all the rows in the table
    rows = table.find_elements(By.TAG_NAME, "tr")

    # Loop through each row and extract the second and fifth columns
    data_pairs = []
    for row in rows[1:]:  # Skip the header row
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) >= 5:  # Ensure there are enough columns in the row
            pool_holder = cols[1].text.strip()  # Second column
            pool_number = cols[4].text.strip()  # Fifth column
            data_pairs.append((pool_holder, pool_number))
    return data_pairs

In [121]:
def get_pool_data(driver, pool_number, holder, file_path_1, file_path_2):
    browse_menu = driver.find_element(By.XPATH, "/html/body/div/nav/ul/li[1]")

    # Create an action chain to hover over the "Browse" menu to reveal the submenu
    action = ActionChains(driver)
    action.move_to_element(browse_menu).perform()
    sleep(1)
    # Now, after hovering, find the "Pool" link
    pool_link = driver.find_element(
        By.XPATH, "//a[contains(@href, 'pfg_view_pool_opt')]"
    )
    sleep(1)
    # Click the "Pool" link
    pool_link.click()
    sleep(3)
    # <input type="text" name="searchtext" class="customSearchText">
    # add pool to search
    search = driver.find_element(By.NAME, "searchtext")
    search.send_keys(pool_number)
    # <input type="submit" class="button" onclick="document.getElementById('type').value='1';" name="searchType" value="Pool Number">
    searchType = driver.find_element(By.NAME, "searchType")
    searchType.click()
    sleep(2)
    loans_button = driver.find_element(By.ID, "loansBtn")
    loans_button.click()
    sleep(2)
    # Locate the table element by its summary attribute
    loan_window = driver.find_element(By.ID, "loanWindow")
    table = loan_window.find_element(By.XPATH, "//table[@summary='Pool Information']")

    # Find all rows in the table body
    rows = table.find_elements(By.XPATH, ".//tbody/tr")

    # Initialize an empty list to hold the table data
    data = []

    # Loop through each row and extract the text of each cell
    for row in rows:
        # Find all cell data in the current row
        cols = row.find_elements(By.TAG_NAME, "td")

        # Append the text content of each column to the data list
        data.append([col.text for col in cols])

    # Define the column headers, excluding the top header row
    columns = [
        "SBA Loan Number",
        "Maturity Date",
        "Net Rate",
        "Originator Rate",
        "Outstanding Balance",
    ]

    # Create a pandas DataFrame
    df = pd.DataFrame(data, columns=columns)
    df["Pool Number"] = pool_number
    df["Holder"] = holder

    # Append if the file exists, else write the DataFrame with headers
    if os.path.exists(file_path_1):
        df.to_csv(file_path_1, mode="a", index=False, header=False)
    else:
        df.to_csv(file_path_1, mode="w", index=False, header=True)

    close_button = driver.find_element(By.CSS_SELECTOR, ".ui-dialog-titlebar-close")
    close_button.click()
    sleep(2)
    try:
        pre_button = driver.find_element(By.ID, "prepayBtn")
        pre_button.click()
        sleep(2)

        prepay_window = driver.find_element(By.ID, "prepayWindow")

        # Locate the table inside the modal window by its summary attribute
        table = prepay_window.find_element(
            By.XPATH, ".//table[@summary='Pool Information']"
        )

        # Find all rows in the table body
        rows = table.find_elements(By.XPATH, ".//tbody/tr")

        # Initialize an empty list to hold the table data
        data2 = []

        # Loop through each row and extract the text of each cell
        for row in rows:
            # Find all cell data in the current row
            cols = row.find_elements(By.TAG_NAME, "td")

            # Append the text content of each column to the data list
            data2.append([col.text for col in cols])

        # Define the column headers, excluding the top header row
        columns2 = [
            "Prepayment Date",
            "Prepayment Type",
            "Prepayment Amount",
            "Number of Loans",
            "Amount Not Applied",
            "Prepayment Status",
            "Reversed Date",
            "Reversed Amount",
            "Number of Reversed Loans",
        ]

        # Create a pandas DataFrame
        df2 = pd.DataFrame(data2, columns=columns2)
        df2["Pool Number"] = pool_number
        close = driver.find_element(By.XPATH, "/html/body/div[3]/div[1]/button")
        close.click()
        # Append if the file exists, else write the DataFrame with headers
        if os.path.exists(file_path_2):
            df2.to_csv(file_path_2, mode="a", index=False, header=False)
        else:
            df2.to_csv(file_path_2, mode="w", index=False, header=True)
    except Exception as e:
        print(e)
        print("No prepayment data found for pool number: ", pool_number)

    sleep(2)

In [122]:
load_dotenv()
password = os.getenv("PASS_CAFS")
username = os.getenv("USER_CAFS")
file_path_1 = "loans_remaining.csv"
file_path_2 = "prepayment.csv"
pool = pd.read_csv("pool_numbers.csv")
pool_numbers = pool["pool_numbers"].tolist()
# Set up Chrome options to make the browser run in headless mode (optional)
chrome_options = Options()
chrome_options.add_argument(
    "--headless"
)  # Comment out if you want to see the browser actions
driver = webdriver.Chrome()

driver.implicitly_wait(5)
driver.get(url)

time.sleep(1)
login(username, password, driver)
time.sleep(1)
count = 0
# driver.get(
#    "https://caweb.sba.gov/ftadistapps/dsp_redirect.cfm?display=Factor%20Pages&feature=FTAPLSFACPGSEXT&MaxETranRoleName=FTAPortalInvestor"
# )
df_pairs = pd.read_csv("pairs.csv")
# pairs = get_pool_list(driver)
df_pairs = df_pairs.dropna()
pairs = df_pairs.values.tolist()
pairs = [i[0] for i in pairs]
for character in ["[", "]", '"', "'", "(", "'", ")"]:
    pairs = [i.strip().replace(character, "") for i in pairs]

pairs = [(i[:-6], i[-6:]) for i in pairs if len(i) > 6]
pairs.pop(-1)
while len(pairs) > 0:
    driver.get(url)

    driver.get(
        "https://caweb.sba.gov/ftadistapps/dsp_redirect.cfm?display=Factor%20Pages&feature=FTAPLSFACPGSEXT&MaxETranRoleName=FTAPortalInvestor"
    )

    driver.switch_to.frame("filecontainer")

    for pool_number in pairs:
        try:
            get_pool_data(
                driver, int(pool_number[1]), pool_number[0], file_path_1, file_path_2
            )
            pairs.remove(pool_number)
        except Exception as e:
            print(e)
            count += 1
            driver.refresh()
            driver.switch_to.default_content()
            break
    if count == 10:
        print("high volume of restarts")
        break
    time.sleep(10)

driver.quit()

Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="prepayBtn"]"}
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6938C3AB5+28005]
	(No symbol) [0x00007FF6938283B0]
	(No symbol) [0x00007FF6936C580A]
	(No symbol) [0x00007FF693715A3E]
	(No symbol) [0x00007FF693715D2C]
	(No symbol) [0x00007FF69375EA97]
	(No symbol) [0x00007FF69373BA7F]
	(No symbol) [0x00007FF69375B8B3]
	(No symbol) [0x00007FF69373B7E3]
	(No symbol) [0x00007FF6937075C8]
	(No symbol) [0x00007FF693708731]
	GetHandleVerifier [0x00007FF693BB643D+3118829]
	GetHandleVerifier [0x00007FF693C06C90+3448640]
	GetHandleVerifier [0x00007FF693BFCF0D+3408317]
	GetHandleVerifier [0x00007FF69398A40B+841403]
	(No symbol) [0x00007FF69383340F]
	(No symbol) [0x00007FF69382F484]
	(No symbol) [0x00007FF69382F61D]
	(No symbol) [0x

In [123]:
df_pairs = pd.read_csv("pairs.csv")
# pairs = get_pool_list(driver)
df_pairs = df_pairs.dropna()
pairs = df_pairs.values.tolist()
pairs = [i[0] for i in pairs]
for character in ["[", "]", '"', "'", "(", "'", ")"]:
    pairs = [i.strip().replace(character, "") for i in pairs]

pairs = [(i[:-6], i[-6:]) for i in pairs if len(i) > 6]
pairs.pop(-1)
pairs

[('BMO CAPITAL MARKETS CORP., ', '530500'),
 ('BMO CAPITAL MARKETS CORP., ', '530513'),
 ('BREAN CAPITAL, LLC, ', '530504'),
 ('BREAN CAPITAL, LLC, ', '530508'),
 ('BREAN CAPITAL, LLC, ', '530507'),
 ('CANTOR FITZGERALD & CO., ', '530512'),
 ('FHN FINANCIAL CAPITAL MARKETS, ', '530503'),
 ('FHN FINANCIAL CAPITAL MARKETS, ', '530497'),
 ('FHN FINANCIAL CAPITAL MARKETS, ', '530496'),
 ('RAYMOND JAMES BANK, N.A., ', '530506'),
 ('RAYMOND JAMES BANK, N.A., ', '530505'),
 ('RAYMOND JAMES BANK, N.A., ', '530501'),
 ('RAYMOND JAMES BANK, N.A., ', '530502'),
 ('RAYMOND JAMES BANK, N.A., ', '530499'),
 ('RAYMOND JAMES BANK, N.A., ', '530511'),
 ('STIFEL NICOLAUS & COMPANY, INC., ', '530498'),
 ('STIFEL NICOLAUS & COMPANY, INC., ', '530510'),
 ('STIFEL NICOLAUS & COMPANY, INC., ', '530509'),
 ('STONEWALL INVESTMENTS TENNESSEE INC., ', '540117'),
 ('FHN FINANCIAL CAPITAL MARKETS, ', '130610'),
 ('BMO CAPITAL MARKETS CORP., ', '530530'),
 ('BREAN CAPITAL, LLC, ', '530524'),
 ('BREAN CAPITAL, LLC, 

In [155]:
import pandas as pd

# Load the data
df_loan = pd.read_csv("loans_remaining.csv")
df_pre = pd.read_csv("prepayment.csv")

# Remove any existing 'holder' column in df_pre if it exists
if "holder" in df_pre.columns:
    df_pre = df_pre.drop(columns=["holder"])

# Remove duplicates in df_loan to ensure unique mapping
df_loan_unique = df_loan.drop_duplicates(subset="Pool Number")

# Create a mapping from Pool Number to holder
pool_to_holder = df_loan_unique.set_index("Pool Number")["holder"]

# Use map to match Pool Number in df_pre with the holder, ensuring same size
df_pre["holder"] = df_pre["Pool Number"].map(pool_to_holder)
df_pre.dropna(inplace=True, subset=["holder"])

In [156]:
df_pre.to_csv("prepayment_2024.csv", index=False)